<center><a href="#"><img src = "https://3d-media.pro/images/Logo-DATA-For-Developpement.png" width = 250, align = "center"></a></center>

<h1 style="text-align:center;"> Project: Develop an end-to-end Machine Learning Pipeline </h1> 

<h3 style="text-align:center;">Instructor: Assan Sanogo</h3>

<h3>Project Overview:</h3>
<p>This project is based on a dataset of 7000+ essays graded by English specialists. This data problem is close to a real-world situation as it requires to be cleaned, an EDA must be thoroughly done so that the team can engineer relevant features.</p>
<p>This project is a NLP problem that will be the foundation of an English program used by the company Easy Sailing Language Training. Their ambition is to have a reliable tool to assess new students’ ability to write in English according to the IELTS grading system. In turn it would help prospective students in knowing how much time they need to invest to get to the next level.</p>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
    * [data processing](#dataprocessing)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

### Introduction: Business Problem <a name="introduction"></a>
<p><strong>DETEMLP</strong> is a project that aims to develop an end-to-end pipeline to process essays and output a grade describing the level of English proficiency. This project is based on a dataset of 7000+ essays graded by English specialists. </p>
<p>The goal is to <strong>create a reliable tool to assess new students’ ability to write in English according to the IELTS grading system</strong>. In this project, we’ll be using data processing, data cleaning, and NLP techniques, including the librairie Spacy. If during this trip we struggle with the dataset, we might reframe the problem as a classification problem.</p>
<p>Let’s dive in</p>

### Data<a name="data"></a>
<p>
Our data collection has been really simplify, here the list of our dataset : </p>
<ul>
<li>test_set.tsv</li>
<li>training_set_rel3.tsv</li>
<li>training_set_rel3.xls</li>
<li>training_set_rel3.xlsx</li>
<li>valid_sample_submission_1_column.csv</li>
<li>valid_sample_submission_1_column_no_header.csv</li>
<li>valid_sample_submission_2_column.csv</li>
<li>valid_sample_submission_5_column.csv</li>
<li>valid_set.tsv</li>
<li>valid_set.xls</li>
<li>valid_set.xlsx </li>
</ul>

<p></p>

<i>Data processing<a name="dataprocessing"></a></i>

In [1]:
# python -m spacy download en_core_web_sm
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('vader_lexicon')
# nltk.download('averaged_perceptron_tagger')
# import librairies

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams, pos_tag
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from spellchecker import SpellChecker
from textblob import TextBlob
import spacy
from collections import Counter
from spellchecker import SpellChecker
import language_tool_python

In [2]:
# take a look to valid set
df_valid_set = pd.read_csv("valid_set.tsv",sep="\t",encoding="latin1")
df_valid_set.head()

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid
0,1788,1,"Dear @ORGANIZATION1, @CAPS1 more and more peop...",1788,NaN
1,1789,1,Dear @LOCATION1 Time @CAPS1 me tell you what I...,1789,NaN
2,1790,1,"Dear Local newspaper, Have you been spending a...",1790,NaN
3,1791,1,"Dear Readers, @CAPS1 you imagine how life woul...",1791,NaN
4,1792,1,"Dear newspaper, I strongly believe that comput...",1792,NaN


In [3]:
df_valid_set.describe()

,essay_id,essay_set,domain1_predictionid,domain2_predictionid
count,4218.000000,4218.000000,4218.000000,600.000000
mean,11282.446420,4.123518,13735.433618,7178.000000
std,6173.633131,2.117188,6964.020021,346.698716
min,1788.000000,1.000000,1788.000000,6579.000000
25%,5243.250000,2.000000,7508.500000,6878.500000
50%,10995.500000,4.000000,13995.500000,7178.000000
75%,16852.750000,6.000000,19852.750000,7477.500000
max,21938.000000,8.000000,24938.000000,7777.000000


In [4]:
df_valid_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4218 entries, 0 to 4217
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   essay_id              4218 non-null   int64  
 1   essay_set             4218 non-null   int64  
 2   essay                 4218 non-null   object 
 3   domain1_predictionid  4218 non-null   int64  
 4   domain2_predictionid  600 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 164.9+ KB


In [5]:
# valid sample submission
df_sample_submission = pd.read_csv("valid_sample_submission_5_column.csv",encoding="latin1")
df_sample_submission.head()

,prediction_id,essay_id,essay_set,essay_weight,predicted_score
0,1788,1788,1,1.0,7
1,1789,1789,1,1.0,8
2,1790,1790,1,1.0,9
3,1791,1791,1,1.0,9
4,1792,1792,1,1.0,9


In [6]:
# training set
df_training_set = pd.read_csv("training_set_rel3.tsv",sep="\t",encoding="latin1")
df_training_set.head() 

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_training_set.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
count,12976.000000,12976.000000,12976.000000,12976.000000,128.000000,12976.000000,1800.000000,1800.000000,1800.000000,2292.000000,...,2292.000000,2292.000000,723.000000,723.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,10295.395808,4.179485,4.127158,4.137408,37.828125,6.800247,3.333889,3.330556,3.333889,2.444154,...,2.635689,2.710297,3.777317,3.589212,3.945312,3.890625,4.078125,3.992188,3.843750,3.617188
std,6309.074105,2.136913,4.212544,4.264330,5.240829,8.970705,0.729103,0.726807,0.729103,1.211730,...,1.142566,1.045795,0.689401,0.693256,0.643668,0.630390,0.622535,0.509687,0.538845,0.603417
min,1.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
25%,4438.750000,2.000000,2.000000,2.000000,36.000000,2.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000
50%,10044.500000,4.000000,3.000000,3.000000,40.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
75%,15681.250000,6.000000,4.000000,4.000000,40.000000,8.000000,4.000000,4.000000,4.000000,3.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,21633.000000,8.000000,30.000000,30.000000,50.000000,60.000000,4.000000,4.000000,4.000000,6.000000,...,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,5.000000,5.000000


In [8]:
df_training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12976 non-null  int64  
 1   essay_set       12976 non-null  int64  
 2   essay           12976 non-null  object 
 3   rater1_domain1  12976 non-null  int64  
 4   rater2_domain1  12976 non-null  int64  
 5   rater3_domain1  128 non-null    float64
 6   domain1_score   12976 non-null  int64  
 7   rater1_domain2  1800 non-null   float64
 8   rater2_domain2  1800 non-null   float64
 9   domain2_score   1800 non-null   float64
 10  rater1_trait1   2292 non-null   float64
 11  rater1_trait2   2292 non-null   float64
 12  rater1_trait3   2292 non-null   float64
 13  rater1_trait4   2292 non-null   float64
 14  rater1_trait5   723 non-null    float64
 15  rater1_trait6   723 non-null    float64
 16  rater2_trait1   2292 non-null   float64
 17  rater2_trait2   2292 non-null  

In [9]:
# THE FEATURES HAHAHAHA
def calculate_features(text):
    """
    Calcule une série de fonctionnalités linguistiques à partir d'un texte donné.
    """

    # Préparation
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    word_counts = Counter(words)
    spell = SpellChecker()
    misspelled = spell.unknown(words)

    # 0. Faute d'Orthographe

    spell = SpellChecker(language='en')
    misspelled = spell.unknown(words)
    # Le nombre de mots mal orthographiés est le nombre de fautes d'orthographe
    count_spelling_errors = len(misspelled)

    # 1. Longueur du texte
    text_length = len(text)

    # 2. Nombre de phrases
    num_sentences = len(sentences)

    # 3. Nombre de mots
    num_words = len(words)

    # 4. Taille moyenne des mots
    avg_word_length = sum(len(word) for word in words) / num_words

    # 5. Nombre de mots uniques
    num_unique_words = len(set(words))

    # 6. Fréquence des mots de fonction
    function_words = set(stopwords.words('english'))
    function_word_frequency = sum(word_counts[word] for word in function_words)

    # 7. Fréquence des mots de contenu
    content_words = set(words) - function_words
    content_word_frequency = sum(word_counts[word] for word in content_words)

    # 9. Fréquence des erreurs d'orthographe
    spelling_errors = len(misspelled)

    # 10. Fréquence des erreurs de grammaire


    # 11. Fréquence des mots positifs/négatifs
    # Ceci nécessiterait une liste de mots positifs/négatifs
    # Ici, nous utilisons le module NLTK's Vader qui a une liste intégrée de mots positifs et négatifs
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    positive_word_proportion = sentiment['pos']
    negative_word_proportion = sentiment['neg']

    # 12. Fréquence des entités nommées
    # Ceci est plus complexe à calculer et nécessiterait une bibliothèque spécialisée
    # Ici, nous utilisons la bibliothèque SpaCy qui a un bon support pour l'extraction d'entités nommées
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    named_entities = [ent.text for ent in doc.ents]
    named_entity_frequency = len(named_entities)

    # 13. Fréquence des citations
    quote_frequency = text.count('"') // 2

    # 14. Fréquence des majuscules
    uppercase_frequency = sum(1 for char in text if char.isupper())

    # 15. Fréquence des signes de ponctuation
    punctuation_marks = {'.', ',', '!', '?', ':', ';', '-', '(', ')', '[', ']', '{', '}', '"', "'"}
    punctuation_frequency = sum(text.count(mark) for mark in punctuation_marks)

    # 16. Fréquence des adverbes
    adverb_frequency = sum(1 for word, pos in pos_tag(words) if pos == 'RB')

    # 17. Fréquence des verbes au passé
    past_tense_verb_frequency = sum(1 for word, pos in pos_tag(words) if pos == 'VBD')

    # 18. Fréquence des verbes au présent
    present_tense_verb_frequency = sum(1 for word, pos in pos_tag(words) if pos in ('VB', 'VBG', 'VBP', 'VBZ'))

    # 19. Fréquence des pronoms
    pronoun_frequency = sum(1 for word, pos in pos_tag(words) if pos in ('PRP', 'PRP$', 'WP', 'WP$'))

    # 20. Fréquence des conjonctions de coordination
    coordinating_conjunction_frequency = sum(1 for word, pos in pos_tag(words) if pos == 'CC')

    # 21. Fréquence des adjectifs : Les adjectifs modifient les noms et peuvent indiquer le style d’écriture.
    tagged = pos_tag(word_tokenize(text))
    adjective_frequency = sum(1 for word, pos in tagged if pos == 'JJ')

    # 22. Fréquence des noms propres : Les noms propres sont des noms spécifiques à des personnes, des lieux, etc. Leur fréquence peut indiquer le sujet du texte.
    tagged_np = pos_tag(word_tokenize(text))
    proper_noun_frequency = sum(1 for word, pos in tagged_np if pos == 'NNP')

    # 23. Fréquence des modaux : Les modaux sont des auxiliaires qui expriment la nécessité, la possibilité, la permission, etc. Leur fréquence peut indiquer le ton du texte.
    tagged_mf = pos_tag(word_tokenize(text))
    modal_frequency = sum(1 for word, pos in tagged_mf if pos == 'MD')

    # 24. Fréquence des déterminants : Les déterminants sont des mots qui précèdent les noms pour indiquer leur référence. Leur fréquence peut indiquer le style d’écriture.
    tagged_df = pos_tag(word_tokenize(text))
    determiner_frequency = sum(1 for word, pos in tagged_df if pos == 'DT')

    # 25. Fréquence des prépositions : Les prépositions relient les noms, les pronoms et les phrases à d’autres mots dans une phrase. Leur fréquence peut indiquer la complexité de la structure des phrases.
    tagged_pf = pos_tag(word_tokenize(text))
    preposition_frequency = sum(1 for word, pos in tagged_pf if pos == 'IN')

    # 26. Fréquence des pronoms interrogatifs : Les pronoms interrogatifs introduisent des questions. Leur fréquence peut indiquer si le texte contient beaucoup de questions.
    tagged_ipf = pos_tag(word_tokenize(text))
    interrogative_pronoun_frequency = sum(1 for word, pos in tagged_ipf if pos in ('WP', 'WP$'))

    # 27. Fréquence des adverbes interrogatifs : Les adverbes interrogatifs introduisent des questions. Leur fréquence peut indiquer si le texte contient beaucoup de questions.
    tagged_iaf = pos_tag(word_tokenize(text))
    interrogative_adverb_frequency = sum(1 for word, pos in tagged_iaf if pos == 'WRB')

    # 28. Fréquence des adverbes de négation : Les adverbes de négation expriment une négation. Leur fréquence peut indiquer le ton du texte
    tagged_naf = pos_tag(word_tokenize(text))
    negation_adverb_frequency = sum(1 for word, pos in tagged_naf if word in ('not', 'never', 'no'))

    # 29. Fréquence des adverbes de degré : Les adverbes de degré modifient les adjectifs et les adverbes pour indiquer le degré. Leur fréquence peut indiquer le style d’écriture.
    tagged_daf = pos_tag(word_tokenize(text))
    degree_adverb_frequency = sum(1 for word, pos in tagged_daf if word in ('very', 'too', 'so', 'quite', 'pretty', 'rather', 'somewhat', 'fairly'))

    # 30. Fréquence des adverbes de manière : Les adverbes de manière décrivent la manière dont une action est effectuée. Leur fréquence peut indiquer le style d’écriture.
    tagged_maf = pos_tag(word_tokenize(text))
    manner_adverb_frequency = sum(1 for word, pos in tagged_maf if pos == 'RB' and word.endswith('ly'))

    # 31. Fréquence des pronoms démonstratifs : Les pronoms démonstratifs indiquent des entités spécifiques. Leur fréquence peut indiquer le style d’écriture.
    tagged_dpf = pos_tag(word_tokenize(text))
    demonstrative_pronoun_frequency = sum(1 for word, pos in tagged_dpf if word in ('this', 'that', 'these', 'those'))

    # 32. Fréquence des pronoms possessifs : Les pronoms possessifs indiquent la possession. Leur fréquence peut indiquer le style d’écriture.
    tagged_ppf = pos_tag(word_tokenize(text))
    possessive_pronoun_frequency = sum(1 for word, pos in tagged_ppf if word in ('my', 'your', 'his', 'her', 'its', 'our', 'their'))

    # 33. Fréquence des pronoms réfléchis : Les pronoms réfléchis renvoient à d’autres entités dans la phrase. Leur fréquence peut indiquer le style d’écriture.
    tagged_rpf = pos_tag(word_tokenize(text))
    reflexive_pronoun_frequency = sum(1 for word, pos in tagged_rpf if word in ('myself', 'yourself', 'himself', 'herself', 'itself', 'ourselves', 'yourselves', 'themselves'))

    # 34. Fréquence des pronoms réciproques : Les pronoms réciproques indiquent une action mutuelle. Leur fréquence peut indiquer le style d’écriture.
    tagged_rpf2 = pos_tag(word_tokenize(text))
    reciprocal_pronoun_frequency = sum(1 for word, pos in tagged_rpf2 if word in ('each other', 'one another'))

    # 35. Fréquence des pronoms indéfinis : Les pronoms indéfinis ne renvoient pas à des entités spécifiques. Leur fréquence peut indiquer le style d’écriture.
    tagged_ipf2 = pos_tag(word_tokenize(text))
    indefinite_pronoun_frequency = sum(1 for word, pos in tagged_ipf2 if word in ('all', 'another', 'any', 'anybody', 'anyone', 'anything', 'each', 'everybody', 'everyone', 'everything', 'few', 'many', 'nobody', 'none', 'nothing', 'one', 'several', 'some', 'somebody', 'someone', 'something'))
    
    # 36. Fréquence des adjectifs possessifs : Les adjectifs possessifs indiquent la possession. Leur fréquence peut indiquer le style d’écriture.
    tagged_af = pos_tag(word_tokenize(text))
    possessive_adjective_frequency = sum(1 for word, pos in tagged_af if word in ('my', 'your', 'his', 'her', 'its', 'our', 'their'))

    # 37. Fréquence des adjectifs démonstratifs : Les adjectifs démonstratifs indiquent des entités spécifiques. Leur fréquence peut indiquer le style d’écriture.
    tagged_daf2 = pos_tag(word_tokenize(text))
    demonstrative_adjective_frequency = sum(1 for word, pos in tagged_daf2 if word in ('this', 'that', 'these', 'those'))

    # 38. Fréquence des adjectifs indéfinis : Les adjectifs indéfinis ne renvoient pas à des entités spécifiques. Leur fréquence peut indiquer le style d’écriture.
    tagged_iaf2 = pos_tag(word_tokenize(text))
    indefinite_adjective_frequency = sum(1 for word, pos in tagged_iaf2 if word in ('all', 'another', 'any', 'both', 'each', 'few', 'many', 'several', 'some'))

    # 39. Fréquence des adjectifs interrogatifs : Les adjectifs interrogatifs introduisent des questions. Leur fréquence peut indiquer si le texte contient beaucoup de questions.
    tagged_iaf3 = pos_tag(word_tokenize(text))
    interrogative_adjective_frequency = sum(1 for word, pos in tagged_iaf3 if word in ('which', 'what', 'whose'))

    # 40. Fréquence des adjectifs exclamatifs : Les adjectifs exclamatifs expriment une émotion ou une réaction. Leur fréquence peut indiquer le ton du texte.
    tagged_eaf = pos_tag(word_tokenize(text))
    exclamatory_adjective_frequency = sum(1 for word, pos in tagged_eaf if word in ('what', 'such'))

    # 41. Fréquence des verbes auxiliaires : Les verbes auxiliaires sont utilisés avec d’autres verbes pour indiquer le temps, la voix, l’humeur, etc. Leur fréquence peut indiquer le style d’écriture.
    tagged_avf = pos_tag(word_tokenize(text))
    auxiliary_verb_frequency = sum(1 for word, pos in tagged_avf if word in ('be', 'have', 'do', 'can', 'could', 'may', 'might', 'must', 'shall', 'should', 'will', 'would'))

    # 42. Fréquence des verbes modaux : Les verbes modaux expriment la nécessité, la possibilité, la permission, etc. Leur fréquence peut indiquer le ton du texte.
    tagged_mvf = pos_tag(word_tokenize(text))
    modal_verb_frequency = sum(1 for word, pos in tagged_mvf if word in ('can', 'could', 'may', 'might', 'must', 'shall', 'should', 'will', 'would'))
    
    # 43. Fréquence des adverbes modificateurs : Les adverbes modificateurs modifient les adjectifs et les adverbes pour indiquer le degré. Leur fréquence peut indiquer le style d’écriture.
    tagged_maf2 = pos_tag(word_tokenize(text))
    modifier_adverb_frequency = sum(1 for word, pos in tagged_maf2 if word in ('very', 'too', 'so', 'quite', 'pretty', 'rather', 'somewhat', 'fairly'))

    # 44. Fréquence des adverbes de liaison : Les adverbes de liaison introduisent une phrase ou une clause. Leur fréquence peut indiquer le style d’écriture.
    tagged_laf = pos_tag(word_tokenize(text))
    linking_adverb_frequency = sum(1 for word, pos in tagged_laf if word in ('however', 'therefore', 'consequently', 'meanwhile', 'furthermore', 'otherwise'))

    # 45. Fréquence des adverbes de négation : Les adverbes de négation expriment une négation. Leur fréquence peut indiquer le ton du texte.
    tagged_naf2 = pos_tag(word_tokenize(text))
    negation_adverb_frequency = sum(1 for word, pos in tagged_naf2 if word in ('not', 'never', 'no'))

    # 46. Fréquence des adverbes de degré : Les adverbes de degré modifient les adjectifs et les adverbes pour indiquer le degré. Leur fréquence peut indiquer le style d’écriture.
    tagged_daf3 = pos_tag(word_tokenize(text))
    degree_adverb_frequency = sum(1 for word, pos in tagged_daf3 if word in ('very', 'too', 'so', 'quite', 'pretty', 'rather', 'somewhat', 'fairly'))

    # 47. Fréquence des adverbes de manière : Les adverbes de manière décrivent la manière dont une action est effectuée. Leur fréquence peut indiquer le style d’écriture.
    tagged_maf3 = pos_tag(word_tokenize(text))
    manner_adverb_frequency = sum(1 for word, pos in tagged_maf3 if pos == 'RB' and word.endswith('ly'))

    # 48. Fréquence des adverbes de lieu : Les adverbes de lieu indiquent le lieu d’une action. Leur fréquence peut indiquer le sujet du texte.
    tagged_paf = pos_tag(word_tokenize(text))
    place_adverb_frequency = sum(1 for word, pos in tagged_paf if word in ('here', 'there', 'everywhere', 'somewhere', 'anywhere', 'nowhere'))

    # 49. Fréquence des conjonctions de coordination : Les conjonctions de coordination relient des mots, des phrases ou des clauses de même importance. Leur fréquence peut indiquer la complexité de la structure des phrases.
    tagged_ccf = pos_tag(word_tokenize(text))
    coordinating_conjunction_frequency = sum(1 for word, pos in tagged_ccf if word in ('and', 'but', 'or', 'so', 'yet', 'for', 'nor'))

    # 50. Fréquence des conjonctions de subordination : Les conjonctions de subordination relient une clause subordonnée à une clause principale. Leur fréquence peut indiquer la complexité de la structure des phrases.
    tagged_scf = pos_tag(word_tokenize(text))
    subordinating_conjunction_frequency = sum(1 for word, pos in tagged_scf if word in ('although', 'because', 'if', 'unless', 'while', 'though', 'whereas', 'as', 'since', 'when', 'until', 'after', 'before'))

    # 51. Fréquence des conjonctions corrélatives : Les conjonctions corrélatives travaillent en paires pour relier des mots, des phrases ou des clauses de même importance. Leur fréquence peut indiquer la complexité de la structure des phrases.
    tagged_ccf2 = pos_tag(word_tokenize(text))
    correlative_conjunction_frequency = sum(1 for word, pos in tagged_ccf2 if word in ('either', 'or', 'neither', 'nor', 'not only', 'but also', 'whether', 'or', 'as', 'as', 'so', 'as'))

    # 52. Fréquence des interjections : Les interjections sont des mots ou des expressions qui expriment une émotion ou une réaction. Leur fréquence peut indiquer le ton du texte.
    tagged_if = pos_tag(word_tokenize(text))
    interjection_frequency = sum(1 for word, pos in tagged_if if pos == 'UH')

    # 53. Fréquence des onomatopées : Les onomatopées sont des mots qui imitent les sons. Leur fréquence peut indiquer le sujet du texte.
    tagged_of = pos_tag(word_tokenize(text))
    onomatopoeia_frequency = sum(1 for word, pos in tagged_of if word in ('bang', 'beep', 'buzz', 'click', 'ding', 'hiss', 'pop', 'sizzle', 'snap', 'whack', 'whoosh', 'zip', 'clank', 'clang', 'creak', 'crunch', 'gulp', 'jingle', 'jangle', 'rattle', 'snore', 'sniff', 'snort', 'squeak', 'swoosh', 'thud', 'toot', 'twang', 'yelp', 'zing', 'boing', 'zap', 'pow', 'zip', 'swish', 'splash', 'crash', 'chirp', 'tweet', 'honk', 'vroom', 'beep', 'zoom', 'giggle', 'murmur', 'moo', 'oink', 'quack'))

    # 54. Fréquence des palindromes : Les palindromes sont des mots qui se lisent de la même manière de gauche à droite et de droite à gauche. Leur fréquence peut indiquer le style d’écriture.
    words = text.split()
    palindrome_frequency = sum(1 for word in words if word == word[::-1])

    return {
        'count_spelling_errors' : count_spelling_errors,
        'text_length': text_length,
        'num_sentences': num_sentences,
        'num_words': num_words,
        'avg_word_length': avg_word_length,
        'num_unique_words': num_unique_words,
        'function_word_frequency': function_word_frequency,
        'content_word_frequency': content_word_frequency,
        'spelling_errors': spelling_errors,
        'quote_frequency': quote_frequency,
        'uppercase_frequency': uppercase_frequency,
        'punctuation_frequency': punctuation_frequency,
        'adverb_frequency': adverb_frequency,
        'past_tense_verb_frequency': past_tense_verb_frequency,
        'present_tense_verb_frequency': present_tense_verb_frequency,
        'pronoun_frequency': pronoun_frequency,
        'coordinating_conjunction_frequency': coordinating_conjunction_frequency,
        'negative_word_proportion' : negative_word_proportion,
        'named_entity_frequency': named_entity_frequency,
        'adjective_frequency': adjective_frequency,
        'proper_noun_frequency' : proper_noun_frequency,
        'modal_frequency' : modal_frequency,
        'determiner_frequency' : determiner_frequency,
        'interrogative_adverb_frequency' : interrogative_adverb_frequency,
        'interrogative_pronoun_frequency' : interrogative_pronoun_frequency,
        'preposition_frequency' : preposition_frequency,
        'negation_adverb_frequency' : negation_adverb_frequency,
        'degree_adverb_frequency' : degree_adverb_frequency,
        'manner_adverb_frequency' : manner_adverb_frequency,
        'demonstrative_pronoun_frequency' : demonstrative_pronoun_frequency,
        'possessive_pronoun_frequency' : possessive_pronoun_frequency,
        'reflexive_pronoun_frequency' : reflexive_pronoun_frequency,
        'reciprocal_pronoun_frequency' : reciprocal_pronoun_frequency,
        'indefinite_pronoun_frequency' : indefinite_pronoun_frequency,
        'possessive_adjective_frequency' : possessive_adjective_frequency,
        'demonstrative_adjective_frequency' : demonstrative_adjective_frequency,
        'indefinite_adjective_frequency' : indefinite_adjective_frequency,
        'interrogative_adjective_frequency' : interrogative_adjective_frequency,
        'exclamatory_adjective_frequency' : exclamatory_adjective_frequency,
        'auxiliary_verb_frequency' : auxiliary_verb_frequency,
        'modal_verb_frequency' : modal_verb_frequency,
        'modifier_adverb_frequency' : modifier_adverb_frequency,
        'linking_adverb_frequency' : linking_adverb_frequency,
        'negation_adverb_frequency' : negation_adverb_frequency,
        'degree_adverb_frequency' : degree_adverb_frequency,
        'manner_adverb_frequency' : manner_adverb_frequency,
        'place_adverb_frequency' : place_adverb_frequency,
        'coordinating_conjunction_frequency' : coordinating_conjunction_frequency,
        'subordinating_conjunction_frequency' : subordinating_conjunction_frequency,
        'correlative_conjunction_frequency' : correlative_conjunction_frequency,
        'interjection_frequency' : interjection_frequency,
        'onomatopoeia_frequency' : onomatopoeia_frequency,
        'palindrome_frequency' : palindrome_frequency,

    }

In [10]:
# verifions un essai
df_training_set['essay'][2]

"Dear, @CAPS1 @CAPS2 @CAPS3 More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. Others have different ideas. A great amount in the world today are using computers, some for work and spme for the fun of it. Computers is one of mans greatest accomplishments. Computers are helpful in so many ways, @CAPS4, news, and live streams. Don't get me wrong way to much people spend time on the computer and they should be out interacting with others but who are we to tell them what to do. When I grow up I want to be a author or a journalist and I know for a fact that both of those jobs involve lots of time on time on the computer, one @MONTH1 spend more time then the other but you know exactly what @CAPS5 getting at. So what if some expert think people are spending to much time on the computer and not exercising, enjoying natures and interacting with family and friend

In [11]:
# test de la fonction
calculate_features(df_training_set['essay'][2])

{'count_spelling_errors': 10,
 'text_length': 1541,
 'num_sentences': 14,
 'num_words': 313,
 'avg_word_length': 4.035143769968051,
 'num_unique_words': 161,
 'function_word_frequency': 132,
 'content_word_frequency': 181,
 'spelling_errors': 10,
 'quote_frequency': 0,
 'uppercase_frequency': 55,
 'punctuation_frequency': 27,
 'adverb_frequency': 11,
 'past_tense_verb_frequency': 1,
 'present_tense_verb_frequency': 52,
 'pronoun_frequency': 21,
 'coordinating_conjunction_frequency': 22,
 'negative_word_proportion': 0.045,
 'named_entity_frequency': 5,
 'adjective_frequency': 15,
 'proper_noun_frequency': 13,
 'modal_frequency': 4,
 'determiner_frequency': 27,
 'interrogative_adverb_frequency': 3,
 'interrogative_pronoun_frequency': 5,
 'preposition_frequency': 32,
 'negation_adverb_frequency': 2,
 'degree_adverb_frequency': 2,
 'manner_adverb_frequency': 1,
 'demonstrative_pronoun_frequency': 6,
 'possessive_pronoun_frequency': 2,
 'reflexive_pronoun_frequency': 0,
 'reciprocal_pronoun

In [43]:
# test prendre un echantillon pour appliquez la fonction
# df_test = df_training_set.sample(20)

In [44]:
# test application de ma fonction 
# result_df2 = df_test['essay'].apply((lambda x: pd.Series(calculate_features(x))))

In [12]:
# Application de la fonction sur le df_training_set
result_df = df_training_set['essay'].apply((lambda x: pd.Series(calculate_features(x))))

In [13]:
# check resultat_df
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 50 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   count_spelling_errors                12976 non-null  int64  
 1   text_length                          12976 non-null  int64  
 2   num_sentences                        12976 non-null  int64  
 3   num_words                            12976 non-null  int64  
 4   avg_word_length                      12976 non-null  float64
 5   num_unique_words                     12976 non-null  int64  
 6   function_word_frequency              12976 non-null  int64  
 7   content_word_frequency               12976 non-null  int64  
 8   ngram_counts                         12976 non-null  object 
 9   spelling_errors                      12976 non-null  int64  
 10  quote_frequency                      12976 non-null  int64  
 11  uppercase_frequency         

In [14]:
result_df.describe()

,count_spelling_errors,text_length,num_sentences,num_words,avg_word_length,num_unique_words,function_word_frequency,content_word_frequency,spelling_errors,quote_frequency,...,auxiliary_verb_frequency,modal_verb_frequency,modifier_adverb_frequency,linking_adverb_frequency,place_adverb_frequency,subordinating_conjunction_frequency,correlative_conjunction_frequency,interjection_frequency,onomatopoeia_frequency,palindrome_frequency
count,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,...,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000
mean,9.471332,1216.011714,12.776896,252.479732,3.963150,119.017494,103.158061,149.321671,9.471332,0.665613,...,9.093866,4.524199,1.595330,0.039535,0.982506,3.886252,2.870916,0.063579,0.039997,9.256936
std,7.911122,958.322588,11.192549,203.361983,0.341754,70.975526,84.125189,122.222254,7.911122,1.659401,...,10.378990,5.229341,2.048094,0.219426,1.549965,3.838976,3.665249,0.288047,0.243330,10.655898
min,0.000000,8.000000,1.000000,2.000000,2.014286,2.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,527.000000,5.000000,109.000000,3.764408,67.000000,45.000000,63.000000,4.000000,0.000000,...,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
50%,8.000000,901.000000,9.000000,184.000000,3.981613,102.000000,73.000000,110.000000,8.000000,0.000000,...,5.000000,3.000000,1.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,5.000000
75%,13.000000,1670.000000,18.000000,348.000000,4.178964,155.000000,143.000000,202.000000,13.000000,1.000000,...,14.000000,7.000000,2.000000,0.000000,1.000000,5.000000,4.000000,0.000000,0.000000,13.000000
max,101.000000,6098.000000,96.000000,1204.000000,15.285714,506.000000,537.000000,925.000000,101.000000,45.000000,...,94.000000,50.000000,26.000000,4.000000,21.000000,36.000000,30.000000,5.000000,5.000000,100.000000


In [12]:
new_training_set = pd.concat([df_training_set,result_df],axis=1)
new_training_set.info()

NameError: name 'result_df' is not defined

In [23]:
new_training_set.describe()

,essay_id,essay_set,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,...,auxiliary_verb_frequency,modal_verb_frequency,modifier_adverb_frequency,linking_adverb_frequency,place_adverb_frequency,subordinating_conjunction_frequency,correlative_conjunction_frequency,interjection_frequency,onomatopoeia_frequency,palindrome_frequency
count,12976.000000,12976.000000,12976.000000,12976.000000,128.000000,12976.000000,1800.000000,1800.000000,1800.000000,2292.000000,...,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000,12976.000000
mean,10295.395808,4.179485,4.127158,4.137408,37.828125,6.800247,3.333889,3.330556,3.333889,2.444154,...,9.093866,4.524199,1.595330,0.039535,0.982506,3.886252,2.870916,0.063579,0.039997,9.256936
std,6309.074105,2.136913,4.212544,4.264330,5.240829,8.970705,0.729103,0.726807,0.729103,1.211730,...,10.378990,5.229341,2.048094,0.219426,1.549965,3.838976,3.665249,0.288047,0.243330,10.655898
min,1.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4438.750000,2.000000,2.000000,2.000000,36.000000,2.000000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
50%,10044.500000,4.000000,3.000000,3.000000,40.000000,3.000000,3.000000,3.000000,3.000000,2.000000,...,5.000000,3.000000,1.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,5.000000
75%,15681.250000,6.000000,4.000000,4.000000,40.000000,8.000000,4.000000,4.000000,4.000000,3.000000,...,14.000000,7.000000,2.000000,0.000000,1.000000,5.000000,4.000000,0.000000,0.000000,13.000000
max,21633.000000,8.000000,30.000000,30.000000,50.000000,60.000000,4.000000,4.000000,4.000000,6.000000,...,94.000000,50.000000,26.000000,4.000000,21.000000,36.000000,30.000000,5.000000,5.000000,100.000000


In [24]:
new_training_set.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,auxiliary_verb_frequency,modal_verb_frequency,modifier_adverb_frequency,linking_adverb_frequency,place_adverb_frequency,subordinating_conjunction_frequency,correlative_conjunction_frequency,interjection_frequency,onomatopoeia_frequency,palindrome_frequency
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,10,4,3,0,1,6,12,0,0,12
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,19,14,4,0,0,6,5,0,1,25


In [25]:
new_training_set.to_csv('new_trainning_set.csv',index=False)

In [26]:
df_test_new_ts = pd.read_csv('new_trainning_set.csv', sep=',', encoding="latin1")

In [27]:
df_test_new_ts.head(3)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,auxiliary_verb_frequency,modal_verb_frequency,modifier_adverb_frequency,linking_adverb_frequency,place_adverb_frequency,subordinating_conjunction_frequency,correlative_conjunction_frequency,interjection_frequency,onomatopoeia_frequency,palindrome_frequency
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,10,4,3,0,1,6,12,0,0,12
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,19,14,4,0,0,6,5,0,1,25
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,12,4,2,0,0,1,3,0,0,13
